In [ ]:
# Get the Pascal VOC dataset
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# pascal_path = 'gdrive/MyDrive/projet/'
# from google.colab import files
# files.upload()  #this will prompt you to upload the kaggle.json
# !pip install -q kaggle
# !mkdir -p ~/.kaggle
# !cp kaggle.json ~/.kaggle/
# !ls ~/.kaggle
# !chmod 600 /root/.kaggle/kaggle.json  # set permission

Saving kaggle(2).json to kaggle(2).json
kaggle.json


In [ ]:
# !kaggle competitions list -s voc

# !kaggle datasets download huanghanchina/pascal-voc-2012 -p "gdrive/MyDrive/projet/"


401 - Unauthorized


In [ ]:
# !ls voc2012/VOC2012/JPEGImages
# !unzip pascal-voc-2012.zip
# !unzip gdrive/MyDrive/projet/pascal-voc-2012.zip
# !pip install tqdm
# !pwd
# !ls /content/voc2012/VOC2012/JPEGImages |  wc -l
# !ls /content/gdrive/MyDrive/projet/
%cd gdrive/MyDrive/projet/
!ls
!unzip archive.zip
# import os
# os.mkdir("/content/gdrive/MyDrive/projet/FFHQSynthPhotos/")

Streaming output truncated to the last 5000 lines.
  inflating: thumbnails128x128/65000.png  
  inflating: thumbnails128x128/65001.png  
  inflating: thumbnails128x128/65002.png  
  inflating: thumbnails128x128/65003.png  
  inflating: thumbnails128x128/65004.png  
  inflating: thumbnails128x128/65005.png  
  inflating: thumbnails128x128/65006.png  
  inflating: thumbnails128x128/65007.png  
  inflating: thumbnails128x128/65008.png  
  inflating: thumbnails128x128/65009.png  
  inflating: thumbnails128x128/65010.png  
  inflating: thumbnails128x128/65011.png  
  inflating: thumbnails128x128/65012.png  
  inflating: thumbnails128x128/65013.png  
  inflating: thumbnails128x128/65014.png  
  inflating: thumbnails128x128/65015.png  
  inflating: thumbnails128x128/65016.png  
  inflating: thumbnails128x128/65017.png  
  inflating: thumbnails128x128/65018.png  
  inflating: thumbnails128x128/65019.png  
  inflating: thumbnails128x128/65020.png  
  inflating: thumbnails128x128/65021.png  
  i

In [ ]:
from random import randrange, choice, uniform
import logging
import os

import cv2
import torchvision
import numpy as np
from google.colab.patches import cv2_imshow
from tqdm import tqdm



def apply_gaussian_noise(image):
    """Adds gausian noise to an image and returns the noisy image

    Args:
        image (array-like): image to alter

    Returns:
        array-like: noisy image
    """

    h, w, c = image.shape
    mu = 0

    # As per the article - select sigma in [5, 50]
    # Check randrange or uniform for article
    sigma = randrange(5, 50)
    noise = np.random.normal(mu, sigma, (h, w, c))

    noisy_image = image + noise

    noisy_img_clipped = noisy_image.clip(min=0, max=255)

    return noisy_img_clipped.astype("uint8")


def apply_gaussian_blur(image):
    """Applies Gaussian blur to image using randomly generated kernel size and sigma value

    Args:
        image (array-like): image to alter

    Returns:
        array-like: altered image
    """
    kval = choice((3, 5, 7))
    ksize = (kval, kval)
    sigma = uniform(1.0, 5.0)

    blurred_image = cv2.GaussianBlur(image, ksize, sigma)

    return blurred_image


def apply_jpeg_compression(image):
    """Compresses the input image with JPEG

    Args:
        image (array-like): image to alter

    Returns:
        array-like: altered image
    """
    quality = randrange(40, 100)
    encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), quality]
    result, encoded_img = cv2.imencode(".jpg", image, encode_param)
    if result == False:
        logging.warning("Image could not be compressed with JPEG")
    decoded_img = cv2.imdecode(encoded_img, 1)

    return decoded_img


def apply_RGB_pos_shift(image):
    """Applies a positional shift on RGB with roll to the image (DEPERECATED)

    Args:
        image (array-like): image to alter

    Returns:
        array-like: altered image
    """

    R = image[:, :, 0]
    G = image[:, :, 1]
    B = image[:, :, 2]

    range_ = 20

    R_shift = randrange(-1 * range_, range_)
    G_shift = randrange(-1 * range_, range_)
    B_shift = randrange(-1 * range_, range_)

    print(R_shift)

    # Unclear wether we should roll at the max value or saturate.
    shifted_img = np.dstack(
        (np.roll(R, R_shift, axis=1), np.roll(G, G_shift, axis=1), np.roll(B, B_shift, axis=1),)
    )

    return shifted_img


def apply_RGB_shift(image):
    """Applies a pixel level value shift on RGB to the image

    *** Can create some artifacts ***

    Args:
        image (array-like): image to alter

    Returns:
        array-like: altered image
    """

    image = image.astype("float32")

    R = image[:, :, 0]
    G = image[:, :, 1]
    B = image[:, :, 2]

    range_ = 20

    R_shift = randrange(-1 * range_, range_)
    G_shift = randrange(-1 * range_, range_)
    B_shift = randrange(-1 * range_, range_)

    shifted_img = np.dstack((R + R_shift, G + G_shift, B + B_shift))

    shifted_img_clipped = shifted_img.clip(0, 255)
    
    return shifted_img_clipped.astype("uint8")


def apply_box_blur(image):
    blurred_image = cv2.blur(image, (3, 3))
    return blurred_image


def apply_all_degradations(image):
    res_im = image
    res_im = apply_gaussian_noise(res_im)
    res_im = apply_gaussian_blur(res_im)
    res_im = apply_jpeg_compression(res_im)
    res_im = apply_RGB_shift(res_im)
    res_im = apply_box_blur(res_im)

    return res_im


# altered = apply_all_degradations(im)

# FFHQ images dir or PASCAL VOC 2012 images dir
images_dir = "/content/gdrive/MyDrive/projet/thumbnails128x128"

# Save directory
save_dir = "/content/gdrive/MyDrive/projet/FFHQSynthPhotos/"
filenames = os.listdir(images_dir)
paths = [images_dir + "/" + filename for filename in filenames]
for filename, path in tqdm(zip(filenames, paths)):
  if not os.path.exists(save_dir + filename):
    image = cv2.imread(path)
    image = apply_all_degradations(image)
    cv2.imwrite(save_dir + filename, image)

print(len(os.listdir(save_dir)) == len(filenames))

70000it [20:23, 57.20it/s]


True


anscombe.json		      mnist_test.csv
california_housing_test.csv   mnist_train_small.csv
california_housing_train.csv  README.md
